## TPC Dataset Import
Import data for TPC test from GCS into Snowflake database

### Set Configs

In [1]:
import config, sf, datetime

# In "Dry Run" mode we generate and print SQL queries but not run them.
# You can manually run them in "workbench" if you want
DRY_RUN = False
TEST = 'ds'  # we want to run TPC-H
SIZE = '100GB'  # dataset size to use in test

`Note: that we use configuration data from config.py`

### Start Snowflake WAREHOUSE

In [2]:
# initiate SnowflakeHelper with Test type and dataset size specified
sf_helper = sf.SnowflakeHelper(TEST, SIZE, config)

# start Warehouse
sf_helper.warehouse_start(verbose=True)

Preparing to open connection to Snowflake...
Connection opened.
['customer_address', 'customer_demographics', 'ship_mode', 'time_dim', 'reason', 'income_band', 'item', 'store', 'call_center', 'customer', 'web_site', 'store_returns', 'household_demographics', 'web_page', 'promotion', 'catalog_page', 'inventory', 'catalog_returns', 'web_returns', 'web_sales', 'catalog_sales', 'store_sales']
running query: USE ROLE ACCOUNTADMIN
running query: ALTER WAREHOUSE TEST1 RESUME;
running query: USE WAREHOUSE TEST1
running query: CREATE DATABASE IF NOT EXISTS test_wh_4x_large_ds_100GB
running query: USE DATABASE test_wh_4x_large_ds_100GB


### Setup STAGE: Link to GCS data source and stages files for uploading

In [3]:
# if not sf_helper.is_integrated:
#     # integrate Snoflake with GCS.
integration_id = sf_helper.create_integration(is_dry_run=DRY_RUN)
print(f'integrated with gcs: {integration_id}')

running query: create or replace file format csv_file_format
                     type = csv
                     field_delimiter = '|'
                     skip_header = 0
                     null_if = ('NULL', 'null')
                     empty_field_as_null = true
                     encoding = 'iso-8859-1' 
                     compression = none;
result (Timestamp('2020-05-12 16:05:09.699694'), Timestamp('2020-05-12 16:05:10.076300'), -1, 1, 0.00021089935999999999, [('File format CSV_FILE_FORMAT successfully created.',)])


--done creating named file format


--creating storage integration: "gcs_ds_100GB_integration"
running query: CREATE STORAGE INTEGRATION gcs_ds_100GB_integration TYPE=EXTERNAL_STAGE STORAGE_PROVIDER=GCS ENABLED=TRUE STORAGE_ALLOWED_LOCATIONS=('gcs://tpc-benchmark-5947/');
Error running query """CREATE STORAGE INTEGRATION gcs_ds_100GB_integration TYPE=EXTERNAL_STAGE STORAGE_PROVIDER=GCS ENABLED=TRUE STORAGE_ALLOWED_LOCATIONS=('gcs://tpc-benchmark-5947/');""", 

### Test STAGE

In [4]:
db_files = sf_helper.list_integration(integration_id)
for table, files in db_files.items():
    print(f'{table}:')
    for file in files:
        print(f'\t{file}')
    print(f'\tmissing {sf_helper.gcs_file_range - len(files)} files\n\n')

unknown table!!!! gcs://tpc-benchmark-5947/ds_100GB_date_dim_1_96.dat
unknown table!!!! gcs://tpc-benchmark-5947/ds_100GB_dbgen_version_1_96.dat
unknown table!!!! gcs://tpc-benchmark-5947/ds_100GB_warehouse_1_96.dat


--done listing stage
call_center:
	gcs://tpc-benchmark-5947/ds_100GB_call_center_1_96.dat
	missing 95 files


catalog_page:
	gcs://tpc-benchmark-5947/ds_100GB_catalog_page_1_96.dat
	missing 95 files


catalog_returns:
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_10_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_11_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_12_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_13_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_14_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_15_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_16_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_17_96.dat
	gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_18_96.dat
	gcs://tpc-benc

### Create tables in Snowflake if needed

In [5]:
#if not sf_helper.is_integrated():
sf_helper.create_schema(is_dry_run=DRY_RUN)



--pushing schema: "/home/vagrant/bq_snowflake_benchmark/h/2.18.0_rc2/dbgen/dss.ddl"
running query: -- 
-- Legal Notice 
-- 
-- This document and associated source code (the "Work") is a part of a 
-- benchmark specification maintained by the TPC. 
-- 
-- The TPC reserves all right, title, and interest to the Work as provided 
-- under U.S. and international laws, including without limitation all patent 
-- and trademark rights therein. 
-- 
-- No Warranty 
-- 
-- 1.1 TO THE MAXIMUM EXTENT PERMITTED BY APPLICABLE LAW, THE INFORMATION 
--     CONTAINED HEREIN IS PROVIDED "AS IS" AND WITH ALL FAULTS, AND THE 
--     AUTHORS AND DEVELOPERS OF THE WORK HEREBY DISCLAIM ALL OTHER 
--     WARRANTIES AND CONDITIONS, EITHER EXPRESS, IMPLIED OR STATUTORY, 
--     INCLUDING, BUT NOT LIMITED TO, ANY (IF ANY) IMPLIED WARRANTIES, 
--     DUTIES OR CONDITIONS OF MERCHANTABILITY, OF FITNESS FOR A PARTICULAR 
--     PURPOSE, OF ACCURACY OR COMPLETENESS OF RESPONSES, OF RESULTS, OF 
--     WORKMANLIKE 

result (Timestamp('2020-05-12 16:05:25.129990'), Timestamp('2020-05-12 16:05:25.574249'), -1, 1, 0.00024878504, [('Table TIME_DIM successfully created.',)])
running query: 
create table reason
(
    r_reason_sk               integer               not null,
    r_reason_id               char(16)              not null,
    r_reason_desc             char(100)                     ,
    primary key (r_reason_sk)
);

result (Timestamp('2020-05-12 16:05:25.679556'), Timestamp('2020-05-12 16:05:26.036723'), -1, 1, 0.00020001351999999998, [('Table REASON successfully created.',)])
running query: 
create table income_band
(
    ib_income_band_sk         integer               not null,
    ib_lower_bound            integer                       ,
    ib_upper_bound            integer                       ,
    primary key (ib_income_band_sk)
);

result (Timestamp('2020-05-12 16:05:26.211359'), Timestamp('2020-05-12 16:05:26.450225'), -1, 1, 0.00013376496, [('Table INCOME_BAND successfully create

result (Timestamp('2020-05-12 16:05:28.474567'), Timestamp('2020-05-12 16:05:28.697122'), -1, 1, 0.00012463079999999998, [('Table WEB_SITE successfully created.',)])
running query: 
create table store_returns
(
    sr_returned_date_sk       integer                       ,
    sr_return_time_sk         integer                       ,
    sr_item_sk                integer               not null,
    sr_customer_sk            integer                       ,
    sr_cdemo_sk               integer                       ,
    sr_hdemo_sk               integer                       ,
    sr_addr_sk                integer                       ,
    sr_store_sk               integer                       ,
    sr_reason_sk              integer                       ,
    sr_ticket_number          integer               not null,
    sr_return_quantity        integer                       ,
    sr_return_amt             decimal(7,2)                  ,
    sr_return_tax             decimal(7,2)   

result (Timestamp('2020-05-12 16:05:32.778947'), Timestamp('2020-05-12 16:05:33.159581'), -1, 1, 0.00021315503999999998, [('Table WEB_RETURNS successfully created.',)])
running query: 
create table web_sales
(
    ws_sold_date_sk           integer                       ,
    ws_sold_time_sk           integer                       ,
    ws_ship_date_sk           integer                       ,
    ws_item_sk                integer               not null,
    ws_bill_customer_sk       integer                       ,
    ws_bill_cdemo_sk          integer                       ,
    ws_bill_hdemo_sk          integer                       ,
    ws_bill_addr_sk           integer                       ,
    ws_ship_customer_sk       integer                       ,
    ws_ship_cdemo_sk          integer                       ,
    ws_ship_hdemo_sk          integer                       ,
    ws_ship_addr_sk           integer                       ,
    ws_web_page_sk            integer         

### Import Data from STAGE to target table

In [6]:
for table, files in db_files.items():
    print(f'Starting to import table: {table}')
    for file in sorted(files):
        print(f'\timporting file: {file}')
        sf_helper.import_data(table, file, integration_id)
        print(f'\tfinished @ {datetime.datetime.now().time()}')

Starting to import table: call_center
	importing file: gcs://tpc-benchmark-5947/ds_100GB_call_center_1_96.dat
running query: copy into call_center from 'gcs://tpc-benchmark-5947/ds_100GB_call_center_1_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:05:39.200364'), Timestamp('2020-05-12 16:05:49.278930'), -1, 1, 0.00564399696, [('gcs://tpc-benchmark-5947/ds_100GB_call_center_1_96.dat', 'LOADED', 30, 30, 1, 0, None, None, None, None)])
	finished @ 16:05:49.279677
Starting to import table: catalog_page
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_page_1_96.dat
running query: copy into catalog_page from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_page_1_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:05:49.379383'), Timestamp('2020-05-12 16:05:56.962476'), -1, 1, 0.00424653208, [('gcs://tpc-benchmark-5947/ds_100GB_cat

result (Timestamp('2020-05-12 16:08:04.818285'), Timestamp('2020-05-12 16:08:13.562723'), -1, 1, 0.00489688528, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_22_96.dat', 'LOADED', 150559, 150559, 1, 0, None, None, None, None)])
	finished @ 16:08:13.563575
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_23_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_23_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:08:14.033259'), Timestamp('2020-05-12 16:08:23.396224'), -1, 1, 0.0052432604, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_23_96.dat', 'LOADED', 149927, 149927, 1, 0, None, None, None, None)])
	finished @ 16:08:23.397068
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_24_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_24_96.dat' storage_integrati

result (Timestamp('2020-05-12 16:10:34.116178'), Timestamp('2020-05-12 16:10:42.839204'), -1, 1, 0.00488489456, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_37_96.dat', 'LOADED', 149917, 149917, 1, 0, None, None, None, None)])
	finished @ 16:10:42.839949
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_38_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_38_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:10:42.933941'), Timestamp('2020-05-12 16:10:52.082844'), -1, 1, 0.00512338568, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_38_96.dat', 'LOADED', 149757, 149757, 1, 0, None, None, None, None)])
	finished @ 16:10:52.083600
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_39_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_39_96.dat' storage_integrat

result (Timestamp('2020-05-12 16:13:07.220489'), Timestamp('2020-05-12 16:13:17.494412'), -1, 1, 0.005753396879999999, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_51_96.dat', 'LOADED', 150527, 150527, 1, 0, None, None, None, None)])
	finished @ 16:13:17.495129
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_52_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_52_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:13:17.756379'), Timestamp('2020-05-12 16:13:26.374675'), -1, 1, 0.00482624576, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_52_96.dat', 'LOADED', 150051, 150051, 1, 0, None, None, None, None)])
	finished @ 16:13:26.375463
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_53_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_53_96.dat' storage_i

result (Timestamp('2020-05-12 16:15:38.895127'), Timestamp('2020-05-12 16:15:47.654730'), -1, 1, 0.00490537768, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_66_96.dat', 'LOADED', 149684, 149684, 1, 0, None, None, None, None)])
	finished @ 16:15:47.655532
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_67_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_67_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 16:15:47.979765'), Timestamp('2020-05-12 16:15:56.935794'), -1, 1, 0.005015376239999999, [('gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_67_96.dat', 'LOADED', 149704, 149704, 1, 0, None, None, None, None)])
	finished @ 16:15:56.936647
	importing file: gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_68_96.dat
running query: copy into catalog_returns from 'gcs://tpc-benchmark-5947/ds_100GB_catalog_returns_68_96.dat' storage_i

...

result (Timestamp('2020-05-12 19:04:51.147359'), Timestamp('2020-05-12 19:05:06.418635'), -1, 1, 0.00855191456, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_27_96.dat', 'LOADED', 750752, 750752, 1, 0, None, None, None, None)])
	finished @ 19:05:06.419333
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_28_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_28_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 19:05:06.683993'), Timestamp('2020-05-12 19:05:20.858414'), -1, 1, 0.00793767576, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_28_96.dat', 'LOADED', 749444, 749444, 1, 0, None, None, None, None)])
	finished @ 19:05:20.859158
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_29_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_29_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format

result (Timestamp('2020-05-12 19:08:49.610075'), Timestamp('2020-05-12 19:09:02.867008'), -1, 1, 0.0074238824799999995, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_41_96.dat', 'LOADED', 749338, 749338, 1, 0, None, None, None, None)])
	finished @ 19:09:02.867735
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_42_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_42_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 19:09:03.128534'), Timestamp('2020-05-12 19:09:18.223314'), -1, 1, 0.008453076799999999, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_42_96.dat', 'LOADED', 749961, 749961, 1, 0, None, None, None, None)])
	finished @ 19:09:18.224098
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_43_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_43_96.dat' storage_integration=gcs_ds_100GB_integration file

result (Timestamp('2020-05-12 19:12:35.198490'), Timestamp('2020-05-12 19:12:49.433930'), -1, 1, 0.0079718464, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_56_96.dat', 'LOADED', 749597, 749597, 1, 0, None, None, None, None)])
	finished @ 19:12:49.434546
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_57_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_57_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 19:12:49.537692'), Timestamp('2020-05-12 19:13:02.786715'), -1, 1, 0.007419452879999999, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_57_96.dat', 'LOADED', 750136, 750136, 1, 0, None, None, None, None)])
	finished @ 19:13:02.787461
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_58_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_58_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(

result (Timestamp('2020-05-12 19:16:33.308342'), Timestamp('2020-05-12 19:16:47.149809'), -1, 1, 0.007751221519999999, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_70_96.dat', 'LOADED', 750186, 750186, 1, 0, None, None, None, None)])
	finished @ 19:16:47.150622
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_71_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_71_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 19:16:47.245660'), Timestamp('2020-05-12 19:17:01.515673'), -1, 1, 0.007991207279999999, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_71_96.dat', 'LOADED', 749799, 749799, 1, 0, None, None, None, None)])
	finished @ 19:17:01.516448
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_72_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_72_96.dat' storage_integration=gcs_ds_100GB_integration file_

result (Timestamp('2020-05-12 19:20:20.578189'), Timestamp('2020-05-12 19:20:35.403794'), -1, 1, 0.0083023388, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_85_96.dat', 'LOADED', 749072, 749072, 1, 0, None, None, None, None)])
	finished @ 19:20:35.404625
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_86_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_86_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(format_name=csv_file_format);
result (Timestamp('2020-05-12 19:20:35.870112'), Timestamp('2020-05-12 19:20:50.046426'), -1, 1, 0.007938735839999999, [('gcs://tpc-benchmark-5947/ds_100GB_web_sales_86_96.dat', 'LOADED', 750535, 750535, 1, 0, None, None, None, None)])
	finished @ 19:20:50.047243
	importing file: gcs://tpc-benchmark-5947/ds_100GB_web_sales_87_96.dat
running query: copy into web_sales from 'gcs://tpc-benchmark-5947/ds_100GB_web_sales_87_96.dat' storage_integration=gcs_ds_100GB_integration file_format=(

### Suspend WAREHOUSE

In [7]:
sf_helper.warehouse_suspend()